In [127]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import xmltocsv

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak
import numpy as np


In [128]:
# Convert XML to CSV


# xmltocsv.XMLtoCSV('Summary_A2C.xml')
# xmltocsv.XMLtoCSV('QueueInfo_A2C.xml')
# xmltocsv.XMLtoCSV('VehicleInfo_A2C.xml')
# xmltocsv.XMLtoCSV('Summary_RR.xml')
# xmltocsv.XMLtoCSV('QueueInfo_RR.xml')
# xmltocsv.XMLtoCSV('VehicleInfo_RR.xml')


In [129]:
modelCSVs = {}

modelCSVs["DQN"] = {"Summary" : "Summary_DQN.csv", "QueueInfo" : "QueueInfo_DQN.csv", "VehicleInfo" : "VehicleInfo_DQN.csv"}
modelCSVs["RR"] = {"Summary" : "Summary_RR.csv", "QueueInfo" : "QueueInfo_RR.csv", "VehicleInfo" : "VehicleInfo_RR.csv"}
modelCSVs["A2C"] = {"Summary" : "Summary_A2C.csv", "QueueInfo" : "QueueInfo_A2C.csv", "VehicleInfo" : "VehicleInfo_A2C.csv"}



In [130]:
modelInfos = {}

for model in modelCSVs.keys():
    summary_df = pd.read_csv(modelCSVs[model]["Summary"], delimiter=";")
    timesteps = summary_df["step_time"]
    summary_df["step_meanSpeed"] = summary_df["step_meanSpeed"].replace(-1, 0)
    mean_speeds = summary_df["step_meanSpeed"]

    

    pre_df = pd.read_csv(modelCSVs[model]["QueueInfo"], delimiter=";")

    queue_df = pre_df.groupby("data_timestep").sum()
    queue_df = queue_df.drop(columns="lane_id")
    queue_df = queue_df.reset_index()

    lane_queueing_length = queue_df["lane_queueing_length"]

    lane_queueing_time = queue_df["lane_queueing_time"]

    trips_df = pd.read_csv(modelCSVs[model]["VehicleInfo"], delimiter=";")
    trips_df['tripinfo_averageSpeed'] = trips_df["tripinfo_routeLength"]/trips_df["tripinfo_duration"]

    Total_Cars = len(trips_df)

    Average_Delay = trips_df["tripinfo_timeLoss"].mean()

    Average_Waiting_Time = trips_df["tripinfo_waitingTime"].mean()

    Average_Speed = trips_df["tripinfo_averageSpeed"].mean()

    Average_Queue_Length = lane_queueing_length.mean()


    modelInfos[model] = {"timesteps" : timesteps, "mean_speeds" : mean_speeds, "lane_queueing_length" : lane_queueing_length, "lane_queueing_time" : lane_queueing_time, "Total_Cars" : Total_Cars,"Average_Delay" : Average_Delay, "Average_Waiting_Time" : Average_Waiting_Time, "Average_Speed" : Average_Speed, "Average_Queue_Length" : Average_Queue_Length}    

In [131]:
modelInfos[model]["mean_speeds"]

# Plot timesteps vs mean_speeds skipping over 5 timesteps at a time
# plt.plot(modelInfos["DQN"]["timesteps"][::5], modelInfos["DQN"]["mean_speeds"][::5], label="DQN")

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5395    0.0
5396    0.0
5397    0.0
5398    0.0
5399    0.0
Name: step_meanSpeed, Length: 5400, dtype: float64

In [132]:
with PdfPages("Comparision.pdf") as pdf:


    for model in modelInfos.keys():
        timesteps = np.array(modelInfos[model]["timesteps"])
        mean_speeds = np.array(modelInfos[model]["mean_speeds"])
        
        # Calculate the average every 15-20 timesteps
        averaged_timesteps = []
        averaged_mean_speeds = []
        average_interval = 50  # Adjust this value to change the interval
        
        i = 0
        while i < len(timesteps):
            averaged_timesteps.append(np.mean(timesteps[i:i+average_interval]))
            averaged_mean_speeds.append(np.mean(mean_speeds[i:i+average_interval]))
            i += average_interval
        
        plt.plot(averaged_timesteps, averaged_mean_speeds, label=model)

    plt.xlabel("Timestep (seconds)")
    plt.ylabel("Mean Speed (m/s)")
    plt.title("Mean Speed vs Timestep")
    plt.grid(True)
    plt.legend()
    pdf.savefig()
    plt.close()


    # Plot data_timestep vs lane_queueing_length
    for model in modelInfos.keys():
        timesteps = np.array(modelInfos[model]["timesteps"])
        queueing_length = np.array(modelInfos[model]["lane_queueing_length"])
        
        # Calculate the average every 15-20 timesteps
        averaged_timesteps = []
        averaged_queueing_length = []
        average_interval = 50  # Adjust this value to change the interval
        
        i = 0
        while i < len(timesteps):
            averaged_timesteps.append(np.mean(timesteps[i:i+average_interval]))
            averaged_queueing_length.append(np.mean(queueing_length[i:i+average_interval]))
            i += average_interval
        
        plt.plot(averaged_timesteps, averaged_queueing_length, label=model)

    plt.xlabel("Timestep (seconds)")
    plt.ylabel("Average Queue Length (metres)")
    plt.title("Average Queue Length vs Timestep")
    plt.grid(True)
    plt.legend()
    pdf.savefig()
    plt.close()

    # Plot data_timestep vs lane_queueing_time
    for model in modelInfos.keys():
        timesteps = np.array(modelInfos[model]["timesteps"])
        queueing_time = np.array(modelInfos[model]["lane_queueing_time"])
        
        # Calculate the average every 15-20 timesteps
        averaged_timesteps = []
        averaged_queueing_time = []
        average_interval = 50  # Adjust this value to change the interval
        
        i = 0
        while i < len(timesteps):
            averaged_timesteps.append(np.mean(timesteps[i:i+average_interval]))
            averaged_queueing_time.append(np.mean(queueing_time[i:i+average_interval]))
            i += average_interval
        
        plt.plot(averaged_timesteps, averaged_queueing_time, label=model)

    plt.xlabel("Timestep (seconds)")
    plt.ylabel("Average Waiting Time (seconds)")
    plt.title("Average Waiting Time vs Timestep")
    plt.grid(True)
    plt.legend()
    pdf.savefig()
    plt.close()


In [133]:
data = {}

for model in modelInfos.keys():
    data[model] = [modelInfos[model]["Average_Delay"], modelInfos[model]["Average_Waiting_Time"], modelInfos[model]["Average_Speed"], modelInfos[model]["Average_Queue_Length"], modelInfos[model]["Total_Cars"]]
  
# Creates pandas DataFrame.
df = pd.DataFrame(data, index=['Average Delay',
                               'Average Waiting Time',
                               'Average Speed',
                               'Average Queue Length',
                               'Total Cars'])

df = df.round(2)

In [134]:
df.to_csv("Comparision.csv", sep=";")
df

,DQN,RR,A2C
Average Delay,19.41,39.52,20.60
Average Waiting Time,9.63,29.07,10.62
Average Speed,8.43,6.41,8.33
Average Queue Length,5.82,18.45,6.47
Total Cars,500.00,500.00,500.00
